In [5]:
import sys
sys.path.append('../')

import numpy as np
from sklearn.decomposition import PCA
from dataset.omsignal_dataset import OMSignalDataset
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from torchvision.transforms import Compose
from dataset.preprocessor import Preprocessor
from dataset.segmenter import PairedHeartBeatSegmenter,HeartBeatSegmenter
from dataset.normalize_transform import Normalize
from dataset.to_float_tensor_transform import ToFloatTensorTransform

from scipy.stats import kendalltau, mode
from sklearn.metrics import accuracy_score
from utils.pickle_model import save_model,load_model

import matplotlib.pyplot as plt

In [6]:
TRAIN_UNLABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_UnlabeledData.dat"
TRAIN_LABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_TrainLabeledData.dat"
VALID_LABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/MILA_ValidationLabeledData.dat"

SEG_TRAIN_UNLABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_SegmentedData_Backup_2.dat"
SEG_TRAIN_LABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_Single_TrainLabeledData.dat"
SEG_VALID_LABELED = "/rap/jvb-000-aa/COURS2019/etudiants/submissions/b2pomt3/OMSignal_Single_ValidationLabeledData.dat"

TRAIN_PSEUDOLABELED_PATH = "/rap/jvb-000-aa/COURS2019/etudiants/data/omsignal/myHeartProject/OMSignal_TrainPseudoLabeledData.dat"

SAVE_PATH = "./results/"

In [10]:
transform = Compose([Preprocessor(),
                     PairedHeartBeatSegmenter(230,take_average=True)])


# train_labeled_data = OMSignalDataset(True,True,SEG_TRAIN_LABELED)
# valid_labeled_data = OMSignalDataset(True,True,SEG_VALID_LABELED)

train_labeled_data = OMSignalDataset(False,True,TRAIN_LABELED_PATH,transform )
valid_labeled_data = OMSignalDataset(False,True,VALID_LABELED_PATH,transform )

# train_x = train_labeled_data.data[:,:-4]
# train_y = train_labeled_data.data[:,-4:]

# valid_x = valid_labeled_data.data[:,:-4]
# valid_y = valid_labeled_data.data[:,-4:]

train_x=np.empty((160,230))
train_y=np.empty((160,4))

valid_x=np.empty((160,230))
valid_y=np.empty((160,4))

for i in range(160):
    ecg , target = train_labeled_data[i]
    train_x[i,:] = np.array(ecg)
    train_y[i] = target
    
    ecg2 , target2 = valid_labeled_data[i]
    valid_x[i,:] = np.array(ecg2)
    valid_y[i] = target2

    



In [12]:
pca = PCA(n_components=30)
pca.fit(train_x)

transformed_train_x = pca.transform(train_x)
transformed_valid_x = pca.transform(valid_x)
# transformed_train_x = train_x
# transformed_valid_x = valid_x

In [9]:
model_ID = LinearDiscriminantAnalysis()
model_ID.fit(transformed_train_x, train_y[:,3]) 
y_pred = model_ID.predict(transformed_valid_x)
print("ID: " + str(accuracy_score(valid_y[:,3],y_pred)))

save_model(model_ID,"./results/ID_model.pkl")

ID: 0.7875


In [ ]:
model_PR = SVR(gamma='scale', C=1.0, epsilon=0.2)
model_PR.fit(transformed_train_x, train_y[:,0]) 
y_pred = model_PR.predict(transformed_valid_x)
print("PR: " + str(kendalltau(valid_y[:,0],y_pred)))

In [ ]:
model_RT = SVR(gamma='scale', C=1.0, epsilon=0.2)
model_RT.fit(transformed_train_x, train_y[:,1]) 
y_pred = model_RT.predict(transformed_valid_x)
print("RT: " + str(kendalltau(valid_y[:,1],y_pred)))

In [13]:
model_RR = SVR(gamma='scale', C=1.0, epsilon=0.2)
model_RR.fit(transformed_train_x, train_y[:,2]) 
y_pred = model_RR.predict(transformed_valid_x)
print("RR: " + str(kendalltau(valid_y[:,2],y_pred)))

save_model(model_ID,"./results/RR_model.pkl")

RR: KendalltauResult(correlation=0.7074963643974111, pvalue=3.108226821821746e-40)


In [ ]:
plt.plot(train_x[0,:])

In [ ]:
model_PR = SVR(gamma='scale', C=1.0, epsilon=0.2)
model_PR.fit(transformed_train_x, train_y[:,0]) 
y_pred = model_PR.predict(transformed_valid_x)
print("PR: " + str(kendalltau(valid_y[:,0],y_pred)))